# This notebook is the instruction of generating the datasets for the experiments in the main paper

## environment 

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import time
import torch
torch.set_num_threads(1) 
torch.set_default_dtype(torch.float64)
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib
import matplotlib.pyplot
import matplotlib.pyplot as plt
%matplotlib inline
import math
import torch.optim as optim
from tqdm import tqdm
from torch.nn.parameter import Parameter
import sys
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
from collections import OrderedDict

cpu


In [2]:
path = os.getcwd() 
parent_path = os.path.abspath(os.path.join(path, os.pardir))
sys.path.append(parent_path)

# load DataGene.py

In [3]:
from DataGenerator.DataGene2D import *
n = 10
x_max = np.pi
x_min = 0.0
num_x = 30
T_max = 1000.0
beta = 5000.0
IC = 200
generator = DataGene2D(device=device,n=n,x_max=x_max,x_min=x_min,num_x=num_x,T_max=T_max,beta=beta,IC=IC)

# save data?
save_data = 1

# choose which case to run
data_option = 0
# 0 - stable
# 1 - unstable
# 2 - noise medium
# 3 - forcing
dt=1.0
length = int(T_max/dt)
if data_option == 0 or data_option ==1:
    dataset, traindata, testdata = generator.Plain(dt)
    # unstable case
    if data_option == 1:
        dt = 200 # larger delta t here
        length = int(T_max/dt)
        dt_list = [i for i in xrange(0,int(T_max+dt),int(dt))]
        traindata = traindata[:,:,dt_list]
        testdata = testdata[:,:,dt_list]
    print(dataset.shape, traindata.shape, testdata.shape)
if data_option == 2:
    err_scale = 1e-4 # the level of noise 
    dataset, traindata, testdata = generator.Noise(dt,err_scale)
    print(dataset.shape, traindata.shape, testdata.shape)
if data_option == 3:
    f_scale = 10.0
    dataset, traindata, testdata = generator.Force(dt,f_scale)
    print(dataset.shape, traindata.shape, testdata.shape)

ValueError: operands could not be broadcast together with shapes (200,31,1001,10) (200,31,1001) 

In [ ]:
# Save data if necessary
if save_data == 1:
    
    savedata = OrderedDict()
    
    if data_option == 0 or data_option == 1:
        datafile = 'case%s_dt%.2e.tar' % (data_option,dt)
    if data_option == 2:
        datafile = 'case%s_err%.2e_dt%.2e.tar' \
        % (data_option,err_scale,dt)
    if data_option == 3:
        datafile = 'case%s_force%.2e_dt%.2e.tar' \
        % (data_option,f_scale,dt)
    
    savedata = {
        'data_option': data_option,
        'data': dataset,
        'train': traindata,
        'test': testdata,
    }
    
    torch.save(savedata,datafile)

# Normalization

In [ ]:
traindata[:,30,:]=0.0 
testdata[:,30,:]=0.0
traindata = traindata+1.0
testdata = testdata+1.0


# training dataset

In [ ]:
x_train, y_train = generator.DataForTrain(traindata)
print x_train.shape, y_train.shape
print x_train[150:].shape,y_train[:-150].shape
print torch.sum(torch.abs(x_train[150:] - y_train[:-150])).data.item()

# testing dataset

In [ ]:
# one step ahead prediction
x_one, y_one = generator.DataForTest(testdata,1)

# multi steps ahead prediction
if data_option == 1:
    multi_step = 3
else:
    multi_step = 10
    
x_mul, y_mul = generator.DataForTest(testdata,10)

# sequential prediction (1000-step prediction)
x_sq, y_sq = generator.DataForTest(testdata,length)

y_one = y_one-1.0
y_mul = y_mul-1.0
y_sq= y_sq-1.0

In [ ]:
exit(0)